Chapter 2.a.iv.  Operation reductions
=======

**Operation** is the base class for most of the **Expression** classes defined in the **Prove-It** theory systems.  An **Operation** represents *operator(s)* acting upon *operand(s)*.  $1 + 5 - 1$, $1 < k \leq l$, and $x_1 \cdot \ldots \cdot x_n$ are a few examples of expressions that can be represented by classes derived from **Operation**.

There are essentially two kinds of reduction rules specific to **Operation** expressions that are applied in the process of performing a lambda application (beta reduction).

1. *Operation replacement*: The operation itself may be replaced, rather than just its operator(s) and/or operand(s), in two different ways.
    1. *Explicit operation replacement* is performed by replacing an operator with a **Lambda** expression.  In that case, the operation is replaced by the result of applying thislambda mapping to the operands (an internal beta reduction).  For example, $P(x, y)$ with $P:[(a, b) \mapsto a+b]$ becomes $x + y$.
    2. *Implicit operation replacement* is performed by replacing an operator with a designated **Literal** operator of an **Operation** class.  In that case, the operation is replaced by one of that derived class rather than the **Operation** class. For example, $P(x, y)$ with $P:+$ becomes $x + y$ with the correct `Add` type rather than a generic **Operation** type.
2. *Arity protection*: As an artifact of allowing either a single operand or multiple operands represented by an **ExprTuple**, rather than always having the operand(s) be an **ExprTuple**, we need to treat the following cases as special reduction rules.
    1. The **Operation** has a single operand that is replaced by an **ExprTuple**.  It must become an operation with a single **ExprTuple** operand rather than multiple operands.  For example, $f(x)$ with $x:(a, b, c)$ becomes $f((a, b, c))$ rather than $f(a, b, c)$.
    2. The **Operation** has an **ExprTuple** of multiple operands in an **ExprRange** that collapses to a single operand.  The outer **ExprTuple** must be eliminated consistent with haveing a single operand.  For example, $f(x_1, \ldots, x_n)$ with $n:1$ becomes $f(x_1)$ rather than $f((x_1))$.

In [1]:
import proveit
%begin operation_reductions

## Creating operation expressions

Operation expressions come in various forms.  In any of its forms an **Operation** has two sub-expressions representing its *operator(s)* and *operand(s)* respectively.  Each of these may either be an **ExprTuple** or it represents a singular operator/operand.  There are two main types of formatting for an **Operation**, though any formatting rules may be defined by a class that derives from **Operation**.  These are *infix* and *function* formatting.

The default formatting of an **Operation** class is *infix* notation. For example,

In [2]:
from proveit import Operation
from proveit._common_ import a, b, c
from proveit.number import Add

add_abc_opclass = Operation(Add._operator_, (a, b, c))

add_abc_opclass: a + b + c

Note, however, that this is not the proper way to create an expression representing $a+b+c$ because it is not of type `Add`, we just borrowed the **Literal** operator for the `Add` operation called `Add._operator_`.

In [3]:
add_abc_opclass.__class__

proveit._core_.expression.operation.operation.Operation

Instead, we should use the `Add` constructor.

In [4]:
add_abc = Add(a, b, c)

add_abc: a + b + c

In [5]:
add_abc.__class__

proveit.number.addition.add.Add

There is also formatting available for multiple operators.  For example,

In [6]:
from proveit.number import Less, LessEq, LesserSequence
seq = LesserSequence((Less._operator_, LessEq._operator_), (a, b, c))

seq: a < b <= c

We can see, below, that the `operators` and `operands` are both **ExprTuple** expression in this case.

In [7]:
seq.exprInfo()

,core type,sub-expressions,expression
0,Operation,operators: 1operands: 2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAFQAAAARBAMAAACx0JYtAAAAMFBMVEX///9UVFSqqqrMzMwiIiJE RESYmJgyMjK6urru7u4QEBCIiIh2dnZmZmbc3NwAAABRgo0vAAAAAXRSTlMAQObYZgAAARBJREFU KM9jYMAEnFEODMQC9gsINrczDkXqBiCSXwEuMMcbh8qUe2CqfwJM4K0bDpVbTkLoIzCBh0Uo8nx3 bq+E+ObIBqhQ4e5LYNq0Aq6KZ/dmBoauB+wfwCqvJMBC4PuE+Q+ANGswwjwnhnAGzmUMvAEgnjjc L0w/GfhBFnB7w93McoBBiIH3JwMLSJwToZS3gGH+ARCDzQem9j6IwRXAMB8cQnxwBwCDtR+ij6n2 AURoKYgA6tdnQPUWfwLEFCDgKAQbw/kHrFSBQZzpAUpg8TcAPQAFzKENIOob0AYG1g1sjjyoUcCY wIqIWT5hEHmRgfMyA6f05oc7UCOW8/Yx9MjhOZbNQH8w3QUE4IkQAJpdRPO3bLp9AAAAAElFTkSu QmCC "" style=""display:inline;vertical-align:middle;"" />"
1,ExprTuple,"3, 4","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAADUAAAAUBAMAAADfOGjQAAAAMFBMVEX///9UVFTMzMx2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAAC+aOQFAAAAAXRSTlMAQObYZgAAAO1JREFU KM9jYOC7wIAN5IGI01ilGBgfAIl1WCTYmxh4FjAw8E6A8rMU4FKH24GEOQMDxwMI/8Y8uNTLXhB5 lIHhBYRfYQOXergMTHE5MFSCaB6LAriU9hIIzTyBYS9IagbCH9yboAzeBQxtQEoqAcmJ7QcgDLYG hhCgNmQ5Bt5miCTTB5AcA98M5LBhW/kAKteG7hYGBpaFIK+yNIDdguoHoJbdDmC3WDLA/c7zCybJ J8TAwGnA4MGACLOOA0iauRTgYQZ2BbLcCaSwBgUFclwAw5phH4J7HIhPdYAAJI6Q4xbZl9wgu3Ck iXsMDAAntTiTA0QmgQAAAABJRU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" />"
2,ExprTuple,"5, 6, 7","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAADsAAAAUBAMAAADB8VhjAAAAMFBMVEX///9UVFTMzMx2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAAC+aOQFAAAAAXRSTlMAQObYZgAAAS5JREFU KM91krFKxEAQhv94Xsgm5mInWOURAhZqIVwj2F1jqZAHEA0iHHYRwcoiIHil6wtIGkvlGkGxSF5A id2VV0sKZ3cTmDNcisk3/+7O/rO7wKDEku9UhU8mrD2nLLMqCk98gZvxyRLwuYAg5tk24FRcGE15 9gbMFtzsLGTuEJf0G9xNjozwOL5vC49D9DK8EO5X7tyY+Z0WZrNZ7En4EjckHkPkWrRrBImma1gx 7BSHgKjhGFFIFJGGWsWVuRru5yjCtu2Rbq2fN8NUnBacNW2XeNCtmRqrqbJWxNi0taNgSD6aeYiV tV3AS/yrHgo6Qqv0Mg1iHR8hRIQDcr5x/nMB55ZosgcN2Pois27IDvWkA+/8SqIO0JXgtb2+pAOS PQcf/8FT+y5/TN/AHyMESix9a0nJAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
3,Literal,,
4,Literal,,
5,Variable,,
6,Variable,,
7,Variable,,


When you want *function* formatting, it is best to use the `Function` class that is derived from `Operation`:

In [8]:
from proveit import Function
from proveit._common_ import f, g, x, y, z
Function(f, x)

f(x)

In [9]:
Function(f, (x, y, z))

f(x, y, z)

It only makes sense to use *function* formatting when there is a single operator, so an error is raised if multiple operators are provided.

In [10]:
try:
    Function((f, g), (x, y))
    assert False, "Expected ValueError."
except ValueError as e:
    print("Expected error:", e)    

Expected error: A Function must be given a single `operator`. (f, g) is not a valid `operator`.


## Operation replacement

We will demonstrate the two types of *operation replacement* in the following examples: explicit and implicit.

### Explicit operation replacement

To start our demonstration, let us define a **Lambda** expression which maps a variable $f$ to an expression in which $f$ is used as an operator.

In [11]:
from proveit import Function, Lambda
from proveit.logic import Equals
from proveit._common_ import a, b, c, f, n, x, y, z
ab_substitution = Lambda(f, Equals(Function(f, a), Function(f, b)))

ab_substitution: f -> (f(a) = f(b))

Now we just need to define a **Lambda** expression for replacing $f$.

In [12]:
from proveit.number import Add, Mult
add_y_then_mult_z = Lambda(x, Mult(Add(x, y), z))

add_y_then_mult_z: x -> ((x + y) * z)

In [13]:
ab_substitution.apply(add_y_then_mult_z)

((a + y) * z) = ((b + y) * z)

Note that the $f(a)$ and $f(b)$ operations themselves were replaced according to the `add_y_then_mult_z` lambda mapping applied to $a$ and $b$ respectively.  Let's show an example with multiple operands.

In [14]:
ab_commutation = Lambda(f, Equals(Function(f, [a, b]), Function(f, [b, a])))

ab_commutation: f -> (f(a, b) = f(b, a))

In [15]:
add_then_mult_z = Lambda((x, y), Mult(Add(x, y), z))

add_then_mult_z: (x, y) -> ((x + y) * z)

In [16]:
ab_commutation.apply(add_then_mult_z)

((a + b) * z) = ((b + a) * z)

### Implicit operation replacement

It is typical to derive classes from **Operation** that have a single corresponding *operator* that is a **Literal**.  This operator should be defined as the `_operator_` attribute of the derived class.  This indicates to **Prove-It** that this specific operator is tied to that specific class.  This knowlege is used in *implicit operation replacement*.  When an operator is replaced by `_operator_` of a particular **Operation** class, the **Operation** is reconstucted to by of that particular type.  For example.

In [17]:
a_add_b_commutation = ab_commutation.apply(Add._operator_)

a_add_b_commutation: (a + b) = (b + a)

Is created using the proper `Add` class for constructing the left and right sides of the equation.

In [18]:
a_add_b_commutation.lhs.__class__

proveit.number.addition.add.Add

In [19]:
a_add_b_commutation.rhs.__class__

proveit.number.addition.add.Add

We can do the same using `Mult._operator_`:

In [20]:
a_mult_b_commutation = ab_commutation.apply(Mult._operator_)

a_mult_b_commutation: (a * b) = (b * a)

In [21]:
a_mult_b_commutation.lhs.__class__

proveit.number.multiplication.mult.Mult

When verifying the correctness of a **Prove-It** proof (double-checking the derivation steps that we will discuss in future chapters), the specific type of **Operation** class of an expression is not important.  But this "reduction rule" is important in regards to the way **Prove-It** works as proofs are being generated.

## Arity protection

The arity of an operation, the number of its operands, must always be respected.  That is, in performing a lambda application and applying reduction rules, the number of operands must not change.  This requires special attention simply because a single operand, which is not an **ExprTuple** will not be wrapped in an **ExprTuple** as a convention for the expression DAGs to be more succinct (smaller).

### Protecting 1-arity

When we start with a function with arity 1 and replace its operand with an **ExprTuple**, we must not distinguish the result from a function with that **ExprTuple** as its operands.  We do this by making the new operands be an **ExprTuple** with a single element which is the **ExprTuple** replacing the original operand.  For example, starting with a simple **Lambda** expression ...

In [22]:
from proveit._common_ import fx
operate_with_unary_f = Lambda(x, fx)

operate_with_unary_f: x -> f(x)

In [23]:
operate_with_unary_f.body

f(x)

... whose body is a function-formatted operation with a single operand that is the lambda parameters $x$ ...

In [24]:
operate_with_unary_f.body.exprInfo()

,core type,sub-expressions,expression
0,Operation,operator: 1operand: 2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAACUAAAAUBAMAAAD4luk4AAAAMFBMVEX///8yMjJERETMzMwQEBCY mJgiIiK6urqqqqp2dnZUVFTc3NyIiIhmZmbu7u4AAADOOTK3AAAAAXRSTlMAQObYZgAAAO9JREFU GNNjYICCrVC6G0K9PbuBgU8BKsa1AUyt7G9gYNwAFeMTAJPfnzMwxMLMYJgDIjg+AIlsuNgTEMGZ wMDAnQAXYyxgYGC9knuBgecC0Iwzq2uAGjmBTAb+AwwMTAEMDC8Zwg4kAtWALNlvwMDA1sDAMI9h /oNqoPEgY/qBmAVoyAMGJZB5zCArp0HFGBj+wMW0GCB6GZg/g8TYQXp9GCB29PF8YDgHsYPvOwPY Aew/mD5wA6V4JwDVfoBo4Ftz7mwoyM1AZ3CCQwTht2cMDF2MASAWShgkPQSzmJDDKuY2hAkLU7gk UthHMzAAABcoOI8ge5+JAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
1,Variable,,
2,Variable,,


... we can apply the map to an **ExprTuple** to obtain

In [25]:
from proveit import ExprTuple
abc_on_unary = operate_with_unary_f.apply(ExprTuple(a, b, c))

abc_on_unary: f((a, b, c))

which is distinguished from $f(a, b, c)$ by wrapping $(a, b, c)$ in an extra **ExprTuple**:

In [26]:
abc_on_unary.exprInfo()

,core type,sub-expressions,expression
0,Operation,operator: 1operand: 3,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAFkAAAAUBAMAAAAU44wuAAAAMFBMVEX///8yMjJERETMzMwQEBCY mJgiIiK6urqqqqp2dnZUVFTc3NyIiIhmZmbu7u4AAADOOTK3AAAAAXRSTlMAQObYZgAAAYRJREFU KM+Fk7FLw0AUxr/U1jTVXvsPqJ1FIbubg4sOXXRRoTgIokL/hMw6tJMgKLg4CB2y6CY6uAgtVHFz MEpREEsrIq0o4su9S5qrgh/8ku/lHnf3Xu4ApWPF39pWkJo1FyIHSaiHmWh2ymVIRyUHBlnDjSaM RAORZejZeQKWwfTU1aI9BZJteqwrQpnfWnZDAasADCl6srQIRpFJ3GzWMVyHBKf7Fx4PV2pOsIuq DYsGfZA5B2J5xpzFGKe0RsUnuxMnlkWCSvTBmQ0MOkwjj2vOKXn4YreGlIMkbcwHJSJeZO48bHDO FcSHNGlZ7kCbwU4kexriLWi3yT2M57TscYQ7oelkP0nvalK0yrKhBQZzCKsUXaSyfB46fg2+MvRy wipFBwgbNIGFg7J4pS+HmIQ06QM0baRpBR+zHVnouTpVKWKVWt68pDl9g915arFhM5asQvvzSa/f 4JHZMuT2tFNl/TLBqVq5l1HMZaRe0G+CE7t0y2H0NuTRb7TJ/r1pi8wPUkt0PD+ySGAAAAAASUVO RK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
1,Variable,,
2,ExprTuple,3,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEsAAAAUBAMAAAA3uN37AAAAMFBMVEX///9UVFTMzMx2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAAC+aOQFAAAAAXRSTlMAQObYZgAAAUpJREFU KM+Fkr9KxEAQxr943pG/FzvBKo9wYKEWwjWC3TWWCnkA0SDCYXciWFkEBK80voCksVSuERSL5AWU 2F15taRwMrMxiREc+IVvdie7szMD9FMwf9uRAm+K0uyHSc3TMgH3ih8zw/pPkeCEQmWuX/c2BD0T KhvN6t6zMIdQ2WbDM4fMGcmC/vV0XzbuxjflhWMPHUqIeCS3YCczF5L01yyRJOa+FcEp8o9wST5h H8CIebOXww1YXUDz0aPyEHvkE0YOXTaNCMmARV58lxaMCuvGSLyybCMuSTeuhalL6YBjVbYUt1wS OXN5wqgnJD7Wepy5O6Q8VTz88glbtEBYgXPeQUJrWmqFLIwVvHow6Exil8IIe/Xk8xT6FanpNlhg /Z0eZXpMs1mHLfEiNFs/aAnVejwpuANBS6hBaoylg9/CyoR/hvxD+AbAO2L3K+qrkgAAAABJRU5E rkJggg== "" style=""display:inline;vertical-align:middle;"" />"
3,ExprTuple,"4, 5, 6","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAADsAAAAUBAMAAADB8VhjAAAAMFBMVEX///9UVFTMzMx2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAAC+aOQFAAAAAXRSTlMAQObYZgAAAS5JREFU KM91krFKxEAQhv94Xsgm5mInWOURAhZqIVwj2F1jqZAHEA0iHHYRwcoiIHil6wtIGkvlGkGxSF5A id2VV0sKZ3cTmDNcisk3/+7O/rO7wKDEku9UhU8mrD2nLLMqCk98gZvxyRLwuYAg5tk24FRcGE15 9gbMFtzsLGTuEJf0G9xNjozwOL5vC49D9DK8EO5X7tyY+Z0WZrNZ7En4EjckHkPkWrRrBImma1gx 7BSHgKjhGFFIFJGGWsWVuRru5yjCtu2Rbq2fN8NUnBacNW2XeNCtmRqrqbJWxNi0taNgSD6aeYiV tV3AS/yrHgo6Qqv0Mg1iHR8hRIQDcr5x/nMB55ZosgcN2Pois27IDvWkA+/8SqIO0JXgtb2+pAOS PQcf/8FT+y5/TN/AHyMESix9a0nJAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
4,Variable,,
5,Variable,,
6,Variable,,


### Reducing to 1-arity

This situation can manifest itself in the reverse direction.  We can start with operand(s) of an indeterminant number via an **ExprRange** within an **ExprTuple** and then reduce it to a single operand which should no longer be wrapped in an **ExprTuple**.  For example, 

In [27]:
from proveit.core_expr_types._common_ import x_1_to_n
nary_operation_map = Lambda(n, Function(f, x_1_to_n))

nary_operation_map: n -> f(x_{1}, ..x_{_a}.., x_{n})

In [28]:
nary_operation_map.body

f(x_{1}, ..x_{_a}.., x_{n})

In [29]:
nary_operation_map.body.exprInfo()

,core type,sub-expressions,expression
0,Operation,operator: 1operands: 2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAHQAAAAUBAMAAACuQAROAAAAMFBMVEX///8yMjJERETMzMwQEBCY mJgiIiK6urqqqqp2dnZUVFTc3NyIiIhmZmbu7u4AAADOOTK3AAAAAXRSTlMAQObYZgAAAZZJREFU OMtjYIAC7gAGUgDHBjD19iyQfsRAGlgNJlf2NzAwSJOo9SGI4Pv+nIGB9QKJWjkPgJz9AUiwbSBR K58DSH8CkNjGQCoQBzr1Si7QsWeBnDentp88gNemM6tr5sA4c4GYH6RcFSgxgTWHTYCBWQGn1pcM YQcSYZxAIN5vACQygA5/wPmPfQPz3QScWucxzH9QDeO0AXE/iOEOtJWBCWQhO26tDxiUEBx7IJ4G 1Qp1Oh6tDAx/ULVqMUAcDOQZENDK/BlIxAafASWCMiD2YYAEE8eD8ww8D/Bp7eP5wHCOIdyVwRsS THzfoUFd36DPsBVi63sBoPhHBggDzmP/wfSBO4D7TQLDP6DoTaAAKDGBrH915s3VAIhWtjygUBYo kS2AEWkPGPjWnDsbysDAdYAdlJKEgXECjkdEQgQ7uA8zlz2AsVgMeCcYMPAlMHQxgjMqIvlzgLRO wEzucNZ+BrYKA1DyT3oIEdCGZflrvw4w8GGmxzdwVhUDz1Rwpou5DRFAyeqsmKGLXohoI5gkFjCs GxgAMudnspfQ94QAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
1,Variable,,
2,ExprTuple,3,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAGcAAAAUBAMAAABi2T6lAAAAMFBMVEX////MzMxUVFR2dna6uroQ EBCqqqqIiIgiIiJmZmYyMjJERETu7u6YmJjc3NwAAACgK8IjAAAAAXRSTlMAQObYZgAAAT9JREFU OMtjYOC7wEASyAMRp0nTw8D0AEisI1ETzwIGBt4JJGpiMGdg4HhAqqajDAwvSNXDwOXAUAmkjhc9 KS7AqxBZBfMEhr1ArxnwtnMsYGDcgNvzEBUQwLuAoQ2o9QDzH5YHjDMbcGqCqIBy2BoYQoDmMHCD LGHBrQmqAgIYP4A0MTDwF+DXBFUB19QGouUFCGkCqeCdK14JUQcMCLYD9QzsB/Bpgqjgdb3A+gAc EJYMDP4J+xneQmzi+QVUcn4BEgNMQFS8qWDgP8DAwGnA4MHAcKj8+NwLUOd1AEU5FICMJgYog6MH rkKdwRUUuQJIyQisie0Ahtty4KxAhsmMDAwnkBMsG0gTM6aHDGAMvg8MLYngBMuwDyY04xMwWI9j xhE8sFkUGIovgLMGWibEzMa8aHxukH9Ize73GBgA/31YGfFuXW4AAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
3,ExprRange,lambda_map: 4start_index: 5end_index: 6,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAFoAAAANBAMAAADbDWR+AAAAMFBMVEX///8QEBBmZmaqqqrMzMxE REQyMjIiIiKYmJh2dna6urrc3NxUVFSIiIju7u4AAAA6P00QAAAAAXRSTlMAQObYZgAAANBJREFU KM9jYHh7Z9fNCwz4AJIKvgkctewJDCwBOBVDVYAB5wPOf1wbWJQKcKqGqIDqZGAFGcuFWzVUBRTw X8CvGqoCCvwdCKkGqeDQ8L3EwMD84D4D7wN8qiEqOBY2MG5gWG8Qz7ADYjbfN6Dc+wQkBpiAqNh3 nYH/AcObu281GqAuKX/AwMB+AMioYYAy2OsYYCrOMiyEWwdWzfwAwxl2cJYIgyYL3HEg1ZyYjp4A Y3B/YKgygbHV/wBD6S1mWMPDjusAw80GFLkGDNUcqFwAM81EKjwVshUAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
4,Lambda,parameter: 9body: 7,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAFkAAAAOCAMAAABzXkDSAAAANlBMVEX///8oKCgNDQ1UVFSqqqrM zMwiIiKYmJgyMjIQEBCIiIh2dnZmZmbc3Nzu7u5ERES6uroAAADIdaz2AAAAAXRSTlMAQObYZgAA APFJREFUOMu1U+0OwiAMrPK5AqV9/5e1jZnCnAlG7Q+yQrle7wbAB4EN/hS+fnO71VJxoz3t02Ec E0J0jbHgGjBvOnGWcyzwQyMqACkixLwETFvStfY3yID9MU3RryIEvCZ+FbsZnwNeL3kKEd5JmFZ9 WeXNSknCkzPNYkUeU8FVYJJiasobzyDkqVEYKAwgFV0OL8heV5fBt1PkPGjqCdAoIB0wYtCWdOzX 0Qx3UPaNNGkx/M+sJLoit3S0SouCEeK+h7UhhxiaQz73d5jRKglrSi9W6djo/vBOSVSxnoh+D61P LQinXxGt9zD/2aFXleAGD/AJBgn9rvMAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
5,Literal,,
6,Variable,,
7,IndexedVar,variable: 8index: 9,
8,Variable,,
9,Variable,,


We can `apply` our lambda expression to $1$ and allow the **ExprRange** to reduce, in a manner discussed in the previous chapter, to just a single $x_1$:

In [30]:
from proveit.number import one
f_x1 = nary_operation_map.apply(one)

f_x1: f(x_{1})

In addition to reducing the **ExprRange** to a single element, we must also eliminate the enclosing **ExprTuple** so that we have a single operand consistent with our convention.

In [31]:
f_x1.exprInfo()

,core type,sub-expressions,expression
0,Operation,operator: 1operand: 2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAC4AAAAUBAMAAAAAdhLPAAAAMFBMVEX///8yMjJERETMzMwQEBCY mJgiIiK6urqqqqp2dnZUVFTc3NyIiIhmZmbu7u4AAADOOTK3AAAAAXRSTlMAQObYZgAAAQpJREFU GNNjYICCrQwIwLEBTL09u4GBTwFJnGE1mFzZ38DAuAFZ/CGI4Pv+nIEhFlmYgfMAyLQPQCIbRZzP ASSZwMDAnYAiziDOwMB6JfcCA88FoKIzq2vmQMXnAjE/0DCmAAaGlwxhBxIZGIxA4oFAvN+AgYGt gYFhHsP8B9UMttdA4m1A3A/ELAUMDA8YlEBCZSDCHoinQcUZGP6giGsxQMxhYP4MFwcRPgwQe/t4 PjCcg4oHgrwL8sMFBvYfTB+4A6DiNxkY2EHeZU9g4Ftz7mwozBxhoFJwSCLCASTOl8DQxRgA4iHC rQ0SbkngMGVggoXz3vw14HCOuQ0JQZR40UYwkeORdQMDACv6PvSS2HvYAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
1,Variable,,
2,IndexedVar,variable: 3index: 4,
3,Variable,,
4,Literal,,


In [32]:
%end operation_reductions

# Next chapter: <a href="tutorial04_relabeling.ipynb">ToDo</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>